/databricks-datasets/definitive-guide/data

In [0]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/definitive-guide/data/retail-data/all/*.csv")\
  .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")


Just making a quick change for git demonstration purposes

In [0]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909


+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [0]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070


+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [0]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364


+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [0]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()


+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



In [0]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()


+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [0]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450


+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [0]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310


/databricks/spark/python/pyspark/sql/functions.py:215: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)
+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [0]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()


+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [0]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()


+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609056|47559.391409298754|  218.08095663447796|   218.08115785023418|
+------------------+------------------+--------------------+---------------------+



In [0]:
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()


+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



In [0]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()


+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085635685E-4|             1052.7280543902734|            1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+



In [0]:
from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show()


+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



Don't forget the difference between a list and a set above (a set are the unique values of the list)

In [0]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   536596|      null|    6|
|   537252|      null|    1|
|   538041|      null|    1|
|   537159|     14527|   28|
|   537213|     12748|    6|
|   538191|     15061|   16|
|  C539301|     13496|    1|
+---------+----------+-----+
only showing top 20 rows



In [0]:
# Grouping with Expressions (page 133)

from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()


+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|   536597|  28|             28|
+---------+----+---------------+
only showing top 20 rows



In [0]:
# Grouping with Maps. What's the difference?

df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show()


+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

In [0]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") == "C540850").show()



+---------+---------+--------------------+--------+--------------+---------+----------+-------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+--------------------+--------+--------------+---------+----------+-------+
|  C540850|    21231|SWEETHEART CERAMI...|      -1|1/12/2011 9:41|     1.25|     12665|Germany|
+---------+---------+--------------------+--------+--------------+---------+----------+-------+



### Window Functions begins here

In [0]:
df.show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 2 rows



In [0]:
# Add date-formatted column

from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")


In [0]:
dfWithDate.show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
only showing top 2 rows



In [0]:
# define window ("windowSpec")--over ordered partition based on all preceding rows up to current

from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)


In [0]:
# windowSpec.show(2)

# returns:
# AttributeError: 'WindowSpec' object has no attribute 'show'

In [0]:
# Define max as max(col("Quantitity")) over windowSpec defined above

from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)


In [0]:
# maxPurchaseQuantity.show(2)

# Also not callable

In [0]:
# Include dense rank so as to avoid problems with ties or duplicated rows
# also defines normal rank based on windowSpec

from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)


In [0]:
# purchaseRank.show(2)

# not callable

## The preceding were definitions of columns, basically, which then can be selected from.  Compare to the SQL statement below

### Basically, the whole partitioning/windowing is defined as windowSpec, then every other column is called from that definition

Only now can we do a select to look at returned values (although the original code from the book has some error with the current version of spark)

In [0]:
from pyspark.sql.functions import col
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
# NOTE THE LEGACY TIME PARSER CONFIGURATION I USED TO BE ABLE TO RUN THE COMMAND
# WOULD BE GREAT PRACTICE TO FIGURE OUT HOW TO BUILD THIS SEQUENCE WITH THE CURRENT VERSION

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show(50)


+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

In [0]:
%sql
-- Check it out in SQL

SELECT CustomerId, date, Quantity,
  rank(Quantity) OVER (PARTITION BY CustomerId, date
                       ORDER BY Quantity DESC NULLS LAST
                       ROWS BETWEEN
                         UNBOUNDED PRECEDING AND
                         CURRENT ROW) as rank,
  dense_rank(Quantity) OVER (PARTITION BY CustomerId, date
                             ORDER BY Quantity DESC NULLS LAST
                             ROWS BETWEEN
                               UNBOUNDED PRECEDING AND
                               CURRENT ROW) as dRank,
  max(Quantity) OVER (PARTITION BY CustomerId, date
                      ORDER BY Quantity DESC NULLS LAST
                      ROWS BETWEEN
                        UNBOUNDED PRECEDING AND
                        CURRENT ROW) as maxPurchase
FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId

CustomerId,date,Quantity,rank,dRank,maxPurchase
12346,2011-01-18,-74215,2,2,74215
12346,2011-01-18,74215,1,1,74215
12347,2010-12-07,12,4,4,36
12347,2011-08-02,3,22,9,36
12347,2011-04-07,240,1,1,240
12347,2011-10-31,48,1,1,48
12347,2011-04-07,36,2,2,240
12347,2011-10-31,36,2,2,48
12347,2011-10-31,36,2,2,48
12347,2010-12-07,12,4,4,36


### Grouping Sets
- Only available in SQL (rollup and cube are used for DFs)

In [0]:
# Get the total quantity of stock codes and customers

# First, ditch the nulls
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")


In [0]:
%sql
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC

CustomerId,stockCode,sum(Quantity)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


#### You can do the same thing using a grouping set:

In [0]:
%sql
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC

customerId,stockCode,sum(Quantity)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


#### Great...but imagine you need total number of items, regardless of customer or stock code?
- Impossible with conventional group bys
- Simple with grouping statements: specify an aggregation at that level as well
- Effectively the union of several different groupings

In [0]:
%sql
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
  GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),())
  ORDER BY CustomerId DESC, stockCode DESC

customerId,stockCode,sum(Quantity)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


### Warning
- Grouping sets depend on null values for aggregation levels--if not filtered out, you will get incorrect results
- The same goes for cubes, rollups, and grouping sets

### Rollups

In [0]:
# Define and show the rollup for grand total, and and grand total by date and country


rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()


+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|        France|           449|
|2010-12-01|          null|         26814|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|       Germany|           117|
|2010-12-01|          EIRE|           243|
|2010-12-01|   Netherlands|            97|
|2010-12-01|        Norway|          1852|
|2010-12-01|     Australia|           107|
|2010-12-02|          EIRE|             4|
|2010-12-02|          null|         21023|
|2010-12-02|United Kingdom|         20873|
|2010-12-02|       Germany|           146|
|2010-12-03|          null|         14830|
|2010-12-03|         Spain|           400|
|2010-12-03|      Portugal|            65|
|2010-12-03|United Kingdom|         10439|
|2010-12-03|          EIRE|          2575|
|2010-12-03|   Switzerland|           110|
|2010-12-03|        France|           239|
+----------

In [0]:
rolledUpDF.where("Date IS NULL").show()

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|       5176450|
+----+-------+--------------+



- a null in both rollup columns specifies the grand total across both of thos columns

In [0]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()


In [0]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
